腾讯云COS的连接测试 - 检查COS端配置是否又问题？

In [ ]:
# 测试COS连接
import os, sys 
from dotenv import load_dotenv
load_dotenv()
from qcloud_cos import CosConfig, CosS3Client
config = CosConfig(
    Region='ap-shanghai', 
    SecretId=os.getenv('TENCENT_COS_ACCESS_KEY_ID'), 
    SecretKey=os.getenv('TENCENT_COS_SECRET_ACCESS_KEY'),
)

client = CosS3Client(config)

try:
    response = client.list_buckets()
    print("连接成功:", response)
except Exception as e:
    print("连接失败:", str(e))

Django 文件管理模块开发测试

In [1]:
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist']


INFO 2025-01-31 23:24:38,983 storage 使用 S3Boto3Storage 初始化存储
INFO 2025-01-31 23:24:38,984 storage ✅ 成功初始化 S3Boto3Storage
INFO 2025-01-31 23:24:38,984 storage default_storage 的类型: S3Storage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: storages.backends.s3boto3.S3Boto3Storage
文件default_storage对么？: S3Storage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- drf_spectacular
- rest_framework_simplejwt.token_blacklist


开始测试models.py

In [ ]:
# 导入需要的模型和工具
from django.core.files.base import ContentFile
from django.contrib.auth import get_user_model
from apps.files.models import FileRecord
from django.core.files.uploadedfile import SimpleUploadedFile
import requests
import os
from django.conf import settings

User = get_user_model()

# 创建测试用户（如果不存在）
def get_or_create_test_user():
    try:
        user = User.objects.get(phone='13800138000')
    except User.DoesNotExist:
        user = User.objects.create_user(
            phone='13800138000',
            password='testpass123',
            username='测试用户'
        )
    return user

def test_file_operations():
    """完整的文件操作测试流程"""
    user = get_or_create_test_user()
    
    print("\n1. 测试文件上传")
    # 创建一个测试文件内容
    file_content = b"This is a test file content"
    test_file = SimpleUploadedFile(
        "test_file.txt",
        file_content,
        content_type="text/plain"
    )
    
    # 创建文件记录
    file_record = FileRecord.objects.create(
        name="test_file.txt",
        file=test_file,
        size=len(file_content),
        type='OTHER',
        owner=user,
        created_by=user.phone
    )
    
    print(f"文件已创建: {file_record.name}")
    print(f"文件存储路径: {file_record.file.name}")
    
    # 测试直接URL访问（如果配置了公开访问）
    print("\n2. 测试直接URL访问")
    direct_url = file_record.url
    print(f"直接访问URL: {direct_url}")
    
    # 测试预签名URL
    print("\n3. 测试预签名URL")
    presigned_url = file_record.get_presigned_url(expires=3600)
    print(f"预签名URL: {presigned_url}")
    
    # 验证URL是否可访问
    if presigned_url:
        try:
            response = requests.get(presigned_url,verify=False)
            print(f"URL访问状态码: {response.status_code}")
            if response.status_code == 200:
                print(f"文件内容验证: {response.content == file_content}")
                print(f"文件内容: {response.content}")
            else:
                print(f"访问失败，响应内容: {response.text}")
        except Exception as e:
            print(f"访问URL时出错: {str(e)}")
    
    # 测试文件删除
    print("\n4. 测试文件删除")
    try:
        file_record.delete()
        print("文件记录已删除")
        # 验证文件是否真的从存储中删除
        try:
            deleted_url_response = requests.get(presigned_url)
            print(f"删除后访问状态码: {deleted_url_response.status_code}")  # 应该返回404或403
        except requests.exceptions.RequestException as e:
            print(f"预期的错误 - 文件已被删除: {str(e)}")
    except Exception as e:
        print(f"删除文件时出错: {str(e)}")
    
    return file_record

# 运行测试
test_result = test_file_operations()

开始测试serializers.py


In [ ]:
# 导入必要的库
import os
import django
import uuid
from datetime import datetime
from django.core.files.uploadedfile import SimpleUploadedFile
from rest_framework.request import Request 

# 导入需要测试的模型和序列化器
from django.contrib.auth import get_user_model
from apps.files.models import FileRecord, FileProjectLink
from apps.files.serializers import (
    UserSerializer, 
    FileRecordSerializer, 
    FileProjectLinkSerializer,
    FileRecordCreateSerializer
)
from rest_framework.test import APIRequestFactory

User = get_user_model()
# 创建测试用户
def get_or_create_test_user():
    try:
        user = User.objects.get(phone='13800138000')
    except User.DoesNotExist:
        user = User.objects.create_user(
            phone='13800138000',
            password='testpass123',
            username='测试用户'
        )
    return user

# 创建模拟请求的辅助函数
def create_request(url='/', user=None, presigned=False):
    factory = APIRequestFactory()
    if presigned:
        wsgi_request = factory.get(url + '?presigned=true')
    else:
        wsgi_request = factory.get(url)
    
    if user:
        wsgi_request.user = user
    else:
        # 如果没有提供用户，设置为 AnonymousUser  - 匿名用户 意味着没有登录，报错的原因
        from django.contrib.auth.models import AnonymousUser
        wsgi_request.user = AnonymousUser()    

    # 将 WSGIRequest 转换为 DRF Request，并确保用户信息被保留
    drf_request = Request(wsgi_request)
    drf_request.user = wsgi_request.user  # 显式设置用户
    
    # 将 WSGIRequest 转换为 DRF Request
    return drf_request

# 创建测试用户和基本请求
test_user = get_or_create_test_user()
request = create_request(user=test_user)

# 验证请求中的用户信息
print("Request user:", request.user)  # 调试用，确认用户已正确设置

# 测试 UserSerializer
print("=== 测试 UserSerializer ===")
user_serializer = UserSerializer(test_user)
print(user_serializer.data)

# 创建测试文件
test_file = SimpleUploadedFile(
    "test_file.txt",
    b"This is a test file content",
    content_type="text/plain"
)

# 测试 FileRecordCreateSerializer
print("\n=== 测试 FileRecordCreateSerializer ===")
file_create_data = {
    'name': 'Test File',
    'file': test_file,
    'size': 25,  # 文件内容的字节数
    'type': 'OTHER',
    'mime_type': 'text/plain',
    'metadata': {'test': 'data'},
    'remarks': 'Test remarks'
}

file_create_serializer = FileRecordCreateSerializer(
    data=file_create_data,
    context={'request': request}
)

if file_create_serializer.is_valid():
    file_record = file_create_serializer.save()
    print("文件记录创建成功:", file_record.id)
else:
    print("创建失败:", file_create_serializer.errors)

# 测试 FileRecordSerializer
print("\n=== 测试 FileRecordSerializer ===")
# 测试普通 URL
request_normal = create_request(user=test_user)
serializer_normal = FileRecordSerializer(file_record, context={'request': request_normal})
print("普通 URL:", serializer_normal.data['url'])

# 测试预签名 URL
request_presigned = create_request(user=test_user, presigned=True)
serializer_presigned = FileRecordSerializer(file_record, context={'request': request_presigned})
print("预签名 URL:", serializer_presigned.data['url'])

# 测试 FileProjectLinkSerializer
print("\n=== 测试 FileProjectLinkSerializer ===")
project_id = uuid.uuid4()
link_data = {
    'file_id': file_record.id,
    'project_id': project_id,
    'link_type': 'ATTACHMENT',
    'sort_order': 1
}

link_serializer = FileProjectLinkSerializer(data=link_data)
if link_serializer.is_valid():
    file_link = link_serializer.save()
    print("文件链接创建成功:", file_link.id)
    
    # 测试链接的序列化输出
    link_output = FileProjectLinkSerializer(file_link)
    print("链接详情:", link_output.data)
else:
    print("链接创建失败:", link_serializer.errors)

# 清理测试数据
print("\n=== 清理测试数据 ===")
try:
    file_record.delete()
    test_user.delete()
    print("测试数据清理完成")
except Exception as e:
    print(f"清理数据时出错: {e}")

开始测试views.py


In [ ]:
# 导入必要的库
import os
import django
import uuid
from datetime import datetime
from django.core.files.uploadedfile import SimpleUploadedFile
from rest_framework.test import APIRequestFactory
from rest_framework.request import Request
from django.test import override_settings
from django.core.handlers.wsgi import WSGIRequest

# 导入需要测试的视图、模型和用户
from apps.files.views import FileViewSet
from apps.files.models import FileRecord
from django.contrib.auth import get_user_model

User = get_user_model()

# 创建测试用户
def get_or_create_test_user():
    try:
        user = User.objects.get(phone='13800138000')
    except User.DoesNotExist:
        user = User.objects.create_user(
            phone='13800138000',
            password='testpass123',
            username='测试用户'
        )
    return user

# 创建模拟请求的辅助函数
def create_request(method='get', url='/', user=None, data=None, format='json', **extra):
    factory = APIRequestFactory()
    
    method_map = {
        'get': factory.get,
        'post': factory.post,
        'delete': factory.delete,
        'put': factory.put,
        'patch': factory.patch
    }
    
    request_method = method_map.get(method.lower(), factory.get)
    wsgi_request = request_method(url, data=data, format=format, **extra)
    
    if user:
        wsgi_request.user = user
    else:
        from django.contrib.auth.models import AnonymousUser
        wsgi_request.user = AnonymousUser()
    
    # 将 WSGIRequest 转换为 DRF Request
    drf_request = Request(wsgi_request)
    drf_request.user = wsgi_request.user  # 确保用户信息被保留
    
    # 设置 parsers（对于文件上传等操作可能需要）
    from rest_framework.parsers import JSONParser, MultiPartParser, FormParser
    drf_request.parsers = [JSONParser(), MultiPartParser(), FormParser()]
    
    return drf_request

# 初始化
print("=== 初始化测试环境 ===")
test_user = get_or_create_test_user()
viewset = FileViewSet()

# 1. 测试获取文件列表
print("\n=== 1. 测试获取文件列表 ===")
list_request = create_request('get', '/api/files/', user=test_user)
viewset.request = list_request
viewset.format_kwarg = None

# 设置必要的属性 - action属性在DRF中是ViewSetMixin的属性, DRF自动赋值。
viewset.action = 'list'  # 设置当前动作为 list
viewset.detail = False   # list 动作不是详情视图

# 执行 list action
response = viewset.list(list_request)
print(f"Status Code: {response.status_code}")
print("Files List:", response.data)

In [ ]:
# 2. 测试文件上传
print("\n=== 2. 测试文件上传 ===")
test_file = SimpleUploadedFile(
    "test_file888.txt",
    b"This is a test file content",
    content_type="text/plain"
)

upload_data = {
    'file': test_file,
    'name': 'test_file.txt',
    'type': 'OTHER',
}

upload_request = create_request(
    method='post',
    url='/api/files/',
    user=test_user,
    data=upload_data,
    format='multipart'
)

viewset.request = upload_request
viewset.format_kwarg = None
viewset.action = 'create'
viewset.detail = False

try:
    response = viewset.create(upload_request)
    print(f"Status Code: {response.status_code}")
    print("Upload Response:", response.data)

    # 检查数据库中的记录
    if response.status_code == 201:
        # 获取最新创建的文件记录
        latest_file = FileRecord.objects.latest('created_at')
        print("\nDatabase Record:")
        print(f"ID: {latest_file.id}")
        print(f"Name: {latest_file.name}")
        print(f"Created At: {latest_file.created_at}")
        

except Exception as e:
    print(f"Error during file upload: {e}")
    import traceback
    traceback.print_exc()

# 保存文件ID用于后续测试
#if response.status_code == 201:
#    file_id = response.data['id']
#    print(f"Uploaded file ID: {file_id}")

In [ ]:
# 保存文件ID用于后续测试
if response.status_code == 201:
    file_id = latest_file.id
    print(f"Uploaded file ID: {file_id}")

    # 3. 测试获取单个文件
    print("\n=== 3. 测试获取单个文件 ===")
    detail_request = create_request('get', f'/api/files/{file_id}/', user=test_user)
    viewset.kwargs = {'pk': file_id}
    viewset.request = detail_request
    viewset.action = 'retrieve'
    viewset.detail = True
    
    response = viewset.retrieve(detail_request, pk=file_id)
    print(f"Status Code: {response.status_code}")
    print("File Details:", response.data)

    # 4. 测试删除文件
    print("\n=== 4. 测试删除文件 ===")
    delete_request = create_request('delete', f'/api/files/{file_id}/', user=test_user)
    viewset.kwargs = {'pk': file_id}
    viewset.request = delete_request
    viewset.action = 'destroy'
    viewset.detail = True
    
    response = viewset.destroy(delete_request, pk=file_id)
    print(f"Delete Status Code: {response.status_code}")

In [ ]:
# 5. 再次获取文件列表确认删除
print("\n=== 5. 确认文件已删除 ===")
final_list_request = create_request('get', '/api/files/', user=test_user)
viewset.request = final_list_request
viewset.format_kwarg = None
viewset.action = 'list'
viewset.detail = False

response = viewset.list(final_list_request)
print(f"Status Code: {response.status_code}")
print("Final Files List:", response.data)

# 清理测试数据
print("\n=== 清理测试数据 ===")
try:
    test_user.delete()
    print("测试数据清理完成")
except Exception as e:
    print(f"清理数据时出错: {e}")

print("\n测试完成!")